In [2]:
import os

In [3]:
os.chdir('../')

Update inside entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    des_dir: Path

update configuration manager in src config 

In [7]:
from precision_agri.constants import *
from precision_agri.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,	
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
				
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
		
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            des_dir=config.des_dir 
        )

        return data_ingestion_config


Update components - data_ingsetion.py

In [9]:
import os
import urllib.request as request
import zipfile
from precision_agri import logger
from precision_agri.utils.common import get_size
from precision_agri.entity.config_entity import DataIngestionConfig
from pathlib import Path


class DataIngestion:
		#we get the config from the Dataingestionconfig
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    #we download the file inside the 
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


Update in pipeline

In [10]:
from precision_agri.config.configuration import ConfigurationManager
from precision_agri.components.data_ingestion import DataIngestion
from precision_agri import logger

In [11]:
STAGE_NAME = "Data Ingestion stage"

class DataIngestionTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()



In [13]:


if __name__ == '__main__':
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj = DataIngestionTrainingPipeline()
        obj.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise e

[2023-06-24 20:41:37,720: INFO: 217384239: >>>>>> stage Data Ingestion stage started <<<<<<]
[2023-06-24 20:41:37,734: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-24 20:41:37,740: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-24 20:41:37,742: ERROR: 217384239: yaml file is empty]
Traceback (most recent call last):
  File "c:\users\rajal\desktop\projects\precision_agriculture\src\precision_agri\utils\common.py", line 20, in read_yaml
    return ConfigBox(content)
  File "box\box.py", line 247, in box.box.Box.__init__
box.exceptions.BoxValueError: Cannot extrapolate Box from string

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\rajal\AppData\Local\Temp\ipykernel_57456\217384239.py", line 5, in <module>
    obj.main()
  File "C:\Users\rajal\AppData\Local\Temp\ipykernel_57456\762812553.py", line 8, in main
    config = ConfigurationManager()
  File "c:\users\rajal\d

ValueError: yaml file is empty